In [1]:
import os
import torch
from torch.utils.data import Dataset, random_split
import torchvision
from torchvision import transforms as T
import pandas as pd
from PIL import Image

In [2]:
class FruitsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [3]:
means = [0.8991, 0.7597, 0.6830]
stds = [0.2119, 0.3765, 0.4125]

fruit_transforms = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=means, std=stds)
])

In [4]:
dataset = FruitsDataset(csv_file="data/fruits.csv", root_dir="data/fruits", transform=fruit_transforms)
train_set, test_set = random_split(dataset, [6, 2], generator=torch.Generator().manual_seed(42))

In [5]:
from torch import nn
from torch import optim

model = nn.Sequential(
    nn.Linear(256 * 256 * 3, 2), 
    nn.Softmax(dim=1)
)

## Sieć neuronowa

In [6]:
img, _ = train_set[0]
img_reshaped = img.reshape(-1)

In [7]:
model(img_reshaped)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [8]:
img_reshaped.shape

torch.Size([196608])

In [9]:
img_reshaped = img_reshaped.unsqueeze(0)
img_reshaped.shape

torch.Size([1, 196608])

In [10]:
out = model(img_reshaped)
out

tensor([[0.7441, 0.2559]], grad_fn=<SoftmaxBackward>)

In [11]:
_, index = torch.max(out, dim=1)
index

tensor([0])

In [12]:
model = nn.Sequential(
    nn.Linear(256 * 256 * 3, 2), 
    nn.LogSoftmax(dim=1)
)

In [13]:
criterion = nn.NLLLoss()

In [14]:
img, label = train_set[0]

In [15]:
out = model(img.reshape(-1).unsqueeze(0))

In [16]:
out

tensor([[-0.1974, -1.7196]], grad_fn=<LogSoftmaxBackward>)

In [17]:
label

tensor(0)

In [18]:
criterion(out, label)

IndexError: dimension specified as 0 but tensor has no dimensions

In [19]:
label.shape

torch.Size([])

In [20]:
label.unsqueeze(0)

tensor([0])

In [21]:
label.unsqueeze(0).shape

torch.Size([1])

In [22]:
criterion(out, label.unsqueeze(0))

tensor(0.1974, grad_fn=<NllLossBackward>)

In [23]:
model = nn.Sequential(
    nn.Linear(256 * 256 * 3, 2), 
    nn.LogSoftmax(dim=1)
)

In [24]:
criterion = nn.NLLLoss()

In [25]:
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
n_epochs = 10

for epoch in range(n_epochs):
    
    losses = []
    
    for img, label in train_set:
        
        out = model(img.reshape(-1).unsqueeze(0))
        loss = criterion(out, label.unsqueeze(0))
        
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch: {epoch}; loss: {sum(losses)/len(losses)}")

Epoch: 0; loss: 14.754602114359537
Epoch: 1; loss: 0.0
Epoch: 2; loss: 0.0
Epoch: 3; loss: 0.0
Epoch: 4; loss: 0.0
Epoch: 5; loss: 0.0
Epoch: 6; loss: 0.0
Epoch: 7; loss: 0.0
Epoch: 8; loss: 0.0
Epoch: 9; loss: 0.0


In [27]:
def get_accuracy(model, dataset):
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in dataset:
            scores = model(x.reshape(-1).unsqueeze(0))
            _, prediction = torch.max(scores, dim=1)
            correct += prediction == y
            total += 1

        acc = float(correct) / float(total) * 100
        print(f"accuracy: {acc:.2f}")

In [28]:
get_accuracy(model, train_set)

accuracy: 100.00


In [29]:
get_accuracy(model, test_set)

accuracy: 100.00
